<h1>Long Term Value Stock Analysis</h1>
<h2 style="color:red;">**This project is not investment advice.**</h2>
<p></p>
<div>
    <p>
        With the in-depth analysis, we can evaluate the stock in terms of long term value. <br/>
        Specifically, we use <b>LTM Score, STM Score, and RV Score</b> for evaluation.
    </p>
    <p><b>LTM Score</b> stands for <b>L</b>ong <b>T</b>erm <b>M</b>omentum Score.</p>
    <p><b>STM Score</b> stands for <b>S</b>ong <b>T</b>erm <b>M</b>omentum Score.</p>
    <p><b>RV Score</b> stands for <b>R</b>obust <b>V</b>alue Score.</p>
    <p>
        As you progress in this project, you will find how to compute them.
        For this project, we will use any stock listed in S&P500.
    </p>
</div>

<h3>Please Make an account at IEX Cloud and grab your token!</h3>
<p>Let's first import our python packages.</p>

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math
import requests           #handles REST API(HTTP/HTTPS) calls
IEX_CLOUD_API_TOKEN = "your token here"

<h1>Table of Contents</h1>
<p></p>
<div>
    <ol>
        <li><a href="#1">Get the S&P 500 Stock List</a></li>
        <li><a href="#2">Get the Stock Data</a></li>
        <li><a href="#3">Process the Stock Data</a></li>
        <li><a href="#4">Filter Stocks with High Momentum and RV Score</a></li>
        <li><a href="#5">Set Portfolio Size</a></li>
        <li><a href="#6">Compute the Number of Shares To Buy</a></li>
    </ol>
</div>

<h2 id="1">1. Get the S&P 500 Stock List</h2>

In [2]:
symbols = pd.read_csv('sp_500_stocks.csv')['Ticker'].values

#verifying the data
symbols

array(['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN',
       'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL',
       'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL',
       'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT',
       'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH',
       'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP',
       'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF.B', 'BIIB',
       'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BRK.B',
       'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB',
       'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CF',
       'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA',
       'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG',
       'COO', 'COP', 'COST', 'COTY', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX',
       'CTAS', 'CTL', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CXO

<h2 id="2">2. Get the Stock Data</h2>

In [3]:
batch_base_url = "https://sandbox.iexapis.com/stable"
Types = "quote,stats,advanced-stats"

batch_api_url = lambda base_url, symbols, Types, token : "{base_url}/stock/market/batch?symbols={symbols}&types={Types}&token={token}".format(
    base_url = base_url,
    symbols = symbols,
    Types = Types,
    token = token
)

def chunks(lst, n):
    """
        this function yields successive n-sized chunks from the given lst.
    """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
#making S&P 500 stock lists into chunks of 100 stocks
symbol_strings = [','.join(symbol_group) for symbol_group in list(chunks(symbols, 100))]

For efficiency, we will use batch call instead of single API call for each stock symbol.

In [4]:
columns = [
    'Ticker',
    'Company Name',
    'Price',
    '5 Year Price Return', #Long-term/Short-term Analysis
    '2 Year Price Return',
    '1 Year Price Return',
    '6 Month Price Return',
    '3 Month Price Return',
    '1 Month Price Return',
    '5 Year Price Return Percentile',
    '2 Year Price Return Percentile',
    '1 Year Price Return Percentile',
    '6 Month Price Return Percentile',
    '3 Month Price Return Percentile',
    '1 Month Price Return Percentile',
    'Price-Books Ratio',    #'Price-to-Books Ratio'
    'Price-Sales Ratio',    #'Price-to-Sales Ratio'
    'EV/EBITDA',            #Enterprise Value by Earnings Before Interest, Taxes, Depreciation, and Amortization
    'EV/GP',                #Enterprise Value by Gross Profit
    'Price-Earnings Ratio', #'Price-to-Earnings Ratio'
    'Price-Earnings Ratio Percentile',
    'Price-Books Ratio Percentile',
    'EV/GP Percentile',
    'Price-Sales Ratio Percentile',
    'EV/EBITDA Percentile',
    'LTM Score',            #Long Term Momentum Score             
    'STM Score',            #Short Term Momentum Score       
    'RV Score',             #Robust Value Score
]

NAN, data = np.NaN, {col : [] for col in columns}
for symbol_string in symbol_strings:
    req = requests.get(batch_api_url(
        batch_base_url,
        symbol_string,
        Types,
        IEX_CLOUD_API_TOKEN
    ))
    assert(req.status_code == 200)
    res = req.json()
    for symbol, datum in res.items():
        stats, quote, aStats = datum['stats'], datum['quote'], datum['advanced-stats']
        #Basic Information
        data['Ticker'].append(symbol)
        data['Company Name'].append(stats['companyName'])
        data['Price'].append(quote['latestPrice'])
        
        #For long-term/short-term momentum analysis
        data['5 Year Price Return'].append(stats['year5ChangePercent'])
        data['2 Year Price Return'].append(stats['year2ChangePercent'])
        data['1 Year Price Return'].append(stats['year1ChangePercent'])
        data['6 Month Price Return'].append(stats['month6ChangePercent'])
        data['3 Month Price Return'].append(stats['month3ChangePercent'])
        data['1 Month Price Return'].append(stats['month1ChangePercent'])
        
        data['5 Year Price Return Percentile'].append(NAN)
        data['2 Year Price Return Percentile'].append(NAN)
        data['1 Year Price Return Percentile'].append(NAN)
        data['6 Month Price Return Percentile'].append(NAN)
        data['3 Month Price Return Percentile'].append(NAN)
        data['1 Month Price Return Percentile'].append(NAN)
        data['LTM Score'].append(NAN)
        data['STM Score'].append(NAN)
        
        #For Robust Value analysis
        peRatio, pbRatio, psRatio = quote['peRatio'], aStats['priceToBook'], aStats['priceToSales']
        data['Price-Earnings Ratio'].append(peRatio if peRatio else NAN)
        data['Price-Books Ratio'].append(pbRatio if pbRatio else NAN)
        data['Price-Sales Ratio'].append(psRatio if psRatio else NAN)
        data['RV Score'].append(NAN)
        
        EV, EBITDA, GP = aStats['enterpriseValue'], aStats['EBITDA'], aStats['grossProfit'] 
        if EV:
            if EBITDA:
                data['EV/EBITDA'].append(
                    round(EV / EBITDA, 5)
                )
            else:
                data['EV/EBITDA'].append(NAN)
                    
            if GP:
                data['EV/GP'].append(
                    round(EV / GP, 5)
                )
            else:
                data['EV/GP'].append(NAN)
        else:
            data['EV/EBITDA'].append(NAN) 
            data['EV/GP'].append(NAN)
        
        data['Price-Earnings Ratio Percentile'].append(NAN) 
        data['Price-Books Ratio Percentile'].append(NAN) 
        data['Price-Sales Ratio Percentile'].append(NAN) 
        data['EV/GP Percentile'].append(NAN) 
        data['EV/EBITDA Percentile'].append(NAN)         
df = pd.DataFrame(data)
df.astype({col: 'float64' for col in columns[3:]})
df

,Ticker,Company Name,Price,5 Year Price Return,2 Year Price Return,1 Year Price Return,6 Month Price Return,3 Month Price Return,1 Month Price Return,5 Year Price Return Percentile,...,EV/GP,Price-Earnings Ratio,Price-Earnings Ratio Percentile,Price-Books Ratio Percentile,EV/GP Percentile,Price-Sales Ratio Percentile,EV/EBITDA Percentile,LTM Score,STM Score,RV Score
0,A,Agilent Technologies Inc.,152.97,2.435816,1.234605,0.565945,0.180920,0.119520,0.019526,NaN,...,14.91822,51.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAL,American Airlines Group Inc,21.50,-0.378622,-0.357865,0.888779,0.354668,0.018946,-0.047147,NaN,...,3.02303,-1.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAP,Advance Auto Parts Inc,219.00,0.316110,0.358911,0.449658,0.264539,0.055164,0.056548,NaN,...,2.95350,22.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL,Apple Inc,147.90,5.477461,1.949613,0.522406,0.066694,0.092577,0.102497,NaN,...,18.55345,32.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABBV,Abbvie Inc,118.54,1.323171,0.918465,0.264603,0.079617,0.070137,0.031693,NaN,...,8.49871,41.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,Yum Brands Inc.,122.12,1.029362,0.078970,0.313011,0.092140,-0.001728,0.000089,NaN,...,10.46100,32.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,ZBH,Zimmer Biomet Holdings Inc,164.64,0.337898,0.324315,0.212278,-0.018628,-0.098772,-0.023361,NaN,...,8.17671,60.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502,ZBRA,Zebra Technologies Corp.,534.11,9.563576,2.022160,0.959175,0.321357,0.087972,0.059665,NaN,...,13.29777,44.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
503,ZION,Zions Bancorporation N.A,53.87,0.079138,0.196887,0.587748,0.106273,-0.032866,-0.020067,NaN,...,2.50820,6.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h2 id="3">3. Process the Stock Data</h2>
<p></p>
<div>
    <ol>
        <li>Check if empty data exists</li>
        <li>If exists, fill them with the median value of the corresponding column</li>
        <li>Compute the percentiles</li>
        <li>Compute <b>Long-Term/Short-Term Momentum</b> and <b>RV(Robust Value) Score</b></li>
    </ol>
    <p>
        This project gets <b>LTM(Long Term Momentum)</b> by computing the mean of 
        <ul>
            <li>5 Year Price Return Percentile</li>
            <li>2 Year Price Return Percentile</li>
            <li>1 Year Price Return Percentile</li>
            <li>6 Month Price Return Percentile</li>
        </ul> because these values represents growth rate of relatively long period of time.
        If it is more than 10 percent, it is considered a good value.
        <b>The higher LTM means that it grows steadily and stably.</b>
    </p>
</div>
<div>
    <p>
        This project gets <b>STM(short Term Momentum)</b> by computing the mean of
        <ul>
            <li>3 Month Price Return</li>
            <li>1 Month Price Return</li>
        </ul> because these values represents growth rate of relatively short period of time.
        If it is more than 10 percent, it is considered a good value.
        <b>Higher STM means that it does not slow down.<b/>
    </p>
</div>
<div>
    <p>
        This project gets <b>RV(Robust Value) Score</b> by computing the mean of 
        <ul>
            <li>Price-Earnings Ratio Percentile</li>
            <li>Price-Books Ratio Percentile</li>
            <li>Price-Sales Ratio Percentile</li>
            <li>EV/EBITDA Percentile</li>
            <li>EV/GP Percentile</li>
        </ul> because these values represents the statistical value of the company.
    </p>
</div>

In [5]:
from statistics import mean

#1. Check if empty data exists. If does, fill it with the mean value.
possible_null_columns = [
    '5 Year Price Return',
    '2 Year Price Return',
    '1 Year Price Return',
    '6 Month Price Return',
    '3 Month Price Return',
    '1 Month Price Return',
    'Price-Earnings Ratio', 
    'Price-Books Ratio',    
    'Price-Sales Ratio',    
    'EV/EBITDA',            #Enterprise Value by Earnings Before Interest, Taxes, Depreciation, and Amortization
    'EV/GP',                #Enterprise Value by Gross Profit
]

means = {col: df[col].mean() for col in possible_null_columns} #memoize the mean of target columns
for row in df.index:
    for col in possible_null_columns:  
        val = df.loc[row, col]
        df.loc[row, col] = means[col] if np.isnan(val) else val
        
#verify
df[possible_null_columns][df[possible_null_columns].isnull().any(axis = 1)]

,5 Year Price Return,2 Year Price Return,1 Year Price Return,6 Month Price Return,3 Month Price Return,1 Month Price Return,Price-Earnings Ratio,Price-Books Ratio,Price-Sales Ratio,EV/EBITDA,EV/GP


Once empty data is filled with mean values, the returned dataFrame should hold nothing.
Now, let's compute LTM/STM Score and RV score.

In [6]:
from scipy.stats import percentileofscore as score

#2.Compute Long-Term/Short-Term Momentum and RV(Robust Value) Score
LongTermPeriods = columns[3:7]
ShortTermPeriods = columns[7:9]
RV_score_columns = columns[15:20]

PercentileColumn = lambda col: f"{col} Percentile"
for row in df.index:
    for LongTermPeriod in LongTermPeriods:  #compute the percentile
        df.loc[row, PercentileColumn(LongTermPeriod)] = score(
            df[LongTermPeriod],         #target column
            df.loc[row, LongTermPeriod] #target item to get percentile score
        )
        
    for ShortTermPeriod in ShortTermPeriods: #compute the percentile
        df.loc[row, PercentileColumn(ShortTermPeriod)] = score(
            df[ShortTermPeriod],         
            df.loc[row, ShortTermPeriod]  
        )
    
    for RV_score_column in RV_score_columns: #compute the percentile
        df.loc[row, PercentileColumn(RV_score_column)] = score(
            df[RV_score_column],         
            df.loc[row, RV_score_column]  
        )
 
    #Compute the key scores
    df.loc[row, 'STM Score'] = mean([
        df.loc[row, PercentileColumn(ShortTermPeriod)] for ShortTermPeriod in ShortTermPeriods
    ])
    df.loc[row, 'LTM Score'] = mean([
        df.loc[row, PercentileColumn(LongTermPeriod)] for LongTermPeriod in LongTermPeriods
    ])
    df.loc[row, 'RV Score'] = mean([
        df.loc[row, PercentileColumn(RV_score_column)] for RV_score_column in RV_score_columns
    ])
    
df = df.set_index('Ticker')

Now, let's verify our fully processed data.

In [7]:
#for clear presentation, round long decimal valuess to 100th 
stocks = df[['Company Name', 'Price', 'LTM Score', 'STM Score', 'RV Score']].round(3)
stocks

,Company Name,Price,LTM Score,STM Score,RV Score
Ticker,,,,,
A,Agilent Technologies Inc.,152.97,77.871,70.297,82.158
AAL,American Airlines Group Inc,21.50,46.485,31.188,9.762
AAP,Advance Auto Parts Inc,219.00,52.921,73.069,31.525
AAPL,Apple Inc,147.90,72.426,85.842,79.089
ABBV,Abbvie Inc,118.54,53.614,67.129,62.139
...,...,...,...,...,...
YUM,Yum Brands Inc.,122.12,37.376,41.089,54.574
ZBH,Zimmer Biomet Holdings Inc,164.64,28.119,16.238,63.030
ZBRA,Zebra Technologies Corp.,534.11,94.604,78.812,80.040


<h2 id="4">4. Filter Stocks with High Momentum and RV Score</h2>

In [8]:
stocks = stocks[(stocks['STM Score'] >= 60) & (stocks['LTM Score'] >= 60) & (stocks['RV Score'] >= 75)].sort_values(
    by = 'LTM Score',
    ascending = False
)

#we will sort them by LTM Score for knapsack algorithm in the next step.
stocks

,Company Name,Price,LTM Score,STM Score,RV Score
Ticker,,,,,
FTNT,Fortinet Inc,276.83,97.723,98.614,94.614
ZBRA,Zebra Technologies Corp.,534.11,94.604,78.812,80.040
IDXX,"Idexx Laboratories, Inc.",679.56,93.564,97.030,96.752
INTU,Intuit Inc,523.15,89.307,95.149,90.891
GOOGL,Alphabet Inc,2664.85,88.069,81.089,76.911
PYPL,PayPal Holdings Inc,314.95,87.228,87.327,90.337
MSCI,MSCI Inc,581.03,86.188,95.545,76.752
CPRT,"Copart, Inc.",148.10,84.901,94.158,82.574
IT,"Gartner, Inc.",266.69,83.267,95.248,76.436


I have set threshold for STM/LTM Scores and RV Scores for 60, 60 and 75 respectively arbitrarily.
These filtered stocks are the ones that does not slow down and has steady growth and robust value. 

<h2 id="5">5. Set Portfolio Size</h2>

In [9]:
def portfolio_input():
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
        return val
    except:
        print("That's not a number! Please Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
        
portfolio_size = portfolio_input()
print("portfolio size:", portfolio_size, "USD")

Enter the value of your portfolio:12000
portfolio size: 12000.0 USD


<h2 id="6">6. Compute the Number of Shares To Buy</h2>
<p></p>
<div>
    <p>Using the portfolio size, we can now compute how much stocks we can buy.
        We will floor the number of purchasable shares since we do not allow fractional purchase here. 
    </p>
</div>

In [10]:
NumberOfSharesToBuy = np.floor(portfolio_size / stocks['Price']) #no fractional purchase
stocks['Number of Shares To Buy'] = NumberOfSharesToBuy
stocks.astype({'Number of Shares To Buy': 'int64'})
stocks

,Company Name,Price,LTM Score,STM Score,RV Score,Number of Shares To Buy
Ticker,,,,,,
FTNT,Fortinet Inc,276.83,97.723,98.614,94.614,43.0
ZBRA,Zebra Technologies Corp.,534.11,94.604,78.812,80.040,22.0
IDXX,"Idexx Laboratories, Inc.",679.56,93.564,97.030,96.752,17.0
INTU,Intuit Inc,523.15,89.307,95.149,90.891,22.0
GOOGL,Alphabet Inc,2664.85,88.069,81.089,76.911,4.0
PYPL,PayPal Holdings Inc,314.95,87.228,87.327,90.337,38.0
MSCI,MSCI Inc,581.03,86.188,95.545,76.752,20.0
CPRT,"Copart, Inc.",148.10,84.901,94.158,82.574,81.0
IT,"Gartner, Inc.",266.69,83.267,95.248,76.436,44.0
